<a href="https://colab.research.google.com/github/jylee2930/AI_-Introduction-2025-/blob/main/Image_CNN_Max_pooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##1. 행렬 연산


In [ ]:
##2. 행렬의 이미지표현

import numpy as np
import matplotlib.pyplot as plt
##0,255 영상 보이기
heart = np.array([[1,0,0,1,0,0,1],
                  [0,1,1,0,1,1,0],
                  [0,1,1,1,1,1,0],
                  [1,0,1,1,1,0,1],
                  [1,1,0,1,0,1,1],
                  [1,1,1,0,1,1,1]])

plt.xticks([])
plt.yticks([])
plt.imshow(heart) ##RGB값으로 보이는 함수, cmap = 'gray'
plt.show()

In [ ]:
##빨간색 영상으로 바꿔보자
## 빨간색 = [1,0,0]
height, width = heart.shape
red_heart = np.zeros((height, width, 3))  # RGB 3채널
for i in range(height):
        for j in range(width):
            if heart[i, j] == 1:
                # 빨간색 [R=1, G=0, B=0]
                red_heart[i, j] = [1.0, 0.0, 0.0]
            else:
                # 흰색 [R=1, G=1, B=1]
                red_heart[i, j] = [1.0, 1.0, 1.0]
plt.imshow(red_heart)
plt.show()

In [ ]:
# --- 회전 ---
def rotate(a, k=1, clockwise=True):
    k = k % 4
    return np.rot90(a, k=-k if clockwise else k)

# --- 반전(미러/플립) ---
def flip_lr(a):         # 좌우 반전 (Left-Right)
    return np.fliplr(a)

def flip_ud(a):         # 상하 반전 (Up-Down)
    return np.flipud(a)

def diag_flip(a):       # 주대각선 대칭 (전치)
    return a.T

def anti_diag_flip(a):  # 반대각선 대칭
    return np.rot90(a.T, k=2)

# --- 색(값) 반전: 0<->1 ---
def invert01(a):
    # 원소가 0과 1로만 이루어졌다고 가정
    return 1 - a

In [ ]:
heart = np.array([[1,0,0,1,0,0,1],
                  [0,1,1,0,1,1,0],
                  [0,1,1,1,1,1,0],
                  [1,0,1,1,1,0,1],
                  [1,1,0,1,0,1,1],
                  [1,1,1,0,1,1,1]])
# ---------------- 사용 예시 ----------------
# 시계방향 90도 회전
r90 = rotate(heart, k=1, clockwise=True)
# 반시계방향 180도 회전
r180 = rotate(heart, k=2, clockwise=False)
# 시계방향 280도 회전
r90 = rotate(heart, k=3, clockwise=True)
# 반시계방향 180도 회전
r180 = rotate(heart, k=-2, clockwise=False)

lr   = flip_lr(heart)
ud   = flip_ud(heart)
d1   = diag_flip(heart)
d2   = anti_diag_flip(heart)
inv  = invert01(heart)

In [ ]:
plt.imshow(r90)
plt.show()

In [ ]:
plt.imshow(lr)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
##0~255 흑백영상 보이기
A = np.array([[77,114,140,191],
              [39,56,46,119],
              [61,29,20,30],
              [151,97,43,26]])

plt.xticks([])
plt.yticks([])
plt.imshow(A,cmap = 'gray',vmin=0,vmax=255)
plt.show()


# 합성곱 신경망(CNN)

In [ ]:
##1. 이미지 처리와 필터링
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm


# 1. 간단한 컬러 이미지 생성 (8x8 크기)
def create_sample_image():
    """빨강, 초록, 파랑이 섞인 간단한 테스트 이미지 생성"""
    img = np.zeros((8, 8, 3), dtype=np.uint8)

    # 빨간색 영역
    img[0:4, 0:4] = [255, 0, 0]
    # 초록색 영역
    img[0:4, 4:8] = [0, 255, 0]
    # 파란색 영역
    img[4:8, 0:4] = [0, 0, 255]
    # 노란색 영역 (빨강 + 초록)
    img[4:8, 4:8] = [255, 255, 0]
    return img

 # 원본 이미지 생성
original = create_sample_image()
plt.imshow(original)
plt.show()

In [ ]:
# 2. 마스크를 이용한 색상 변환
def apply_color_mask(image, mask):
    """
    마스크를 이용해 색상 변환
    mask: [R계수, G계수, B계수] 형태
    예: [1.0, 0.5, 0.3]은 빨강은 그대로, 초록은 50%, 파랑은 30%로 감소
    """
    result = image.copy().astype(float)
    result[:, :, 0] *= mask[0]  # Red 채널
    result[:, :, 1] *= mask[1]  # Green 채널
    result[:, :, 2] *= mask[2]  # Blue 채널

    # 0-255 범위로 클리핑
    result = np.clip(result, 0, 255).astype(np.uint8)
    return result

# 3. 컨볼루션을 이용한 다운샘플링 (화소 줄이기)
def apply_convolution_downsampling(image, kernel_size=2):
    """
    컨볼루션(평균 필터)을 이용한 다운샘플링
    kernel_size x kernel_size 영역의 평균을 구해 화소를 줄임
    """
    h, w, c = image.shape
    new_h = h // kernel_size
    new_w = w // kernel_size

    # 결과 이미지
    result = np.zeros((new_h, new_w, c), dtype=np.uint8)

    # 각 채널에 대해 평균 필터 적용
    for i in range(new_h):
        for j in range(new_w):
            # kernel_size x kernel_size 영역의 평균 계산
            region = image[i*kernel_size:(i+1)*kernel_size,
                          j*kernel_size:(j+1)*kernel_size, :]
            result[i, j, :] = np.mean(region, axis=(0, 1))

    return result

In [ ]:
# 원본 이미지 생성
original = create_sample_image()

# 예제 1: 여러 가지 색상 마스크 적용
mask1 = [1.0, 0.5, 0.3]  # 빨강 강조, 초록/파랑 감소
mask2 = [0.3, 1.0, 0.5]  # 초록 강조
mask3 = [0.5, 0.3, 1.0]  # 파랑 강조

masked1 = apply_color_mask(original, mask1)
masked2 = apply_color_mask(original, mask2)
masked3 = apply_color_mask(original, mask3)

# 예제 2: 다운샘플링
downsampled_2x = apply_convolution_downsampling(original, kernel_size=2)

plt.imshow(original)
plt.show()


In [ ]:
plt.imshow(masked1)
plt.show()

###콘볼루션(Convolution, Filter)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO

# 컨볼루션 연산 함수
def apply_convolution(image, kernel):
    """
    이미지에 필터(커널)를 적용하는 함수

    Parameters:
    - image: 입력 이미지 (2D numpy array)
    - kernel: 필터 (2D numpy array)

    Returns:
    - feature_map: 필터 적용 후 특성 맵
    """
    # 이미지와 커널의 크기
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape

    # 출력 이미지 크기 계산
    output_height = image_height - kernel_height + 1
    output_width = image_width - kernel_width + 1

    # 특성 맵 초기화
    feature_map = np.zeros((output_height, output_width))

    # 컨볼루션 연산
    for i in range(output_height):
        for j in range(output_width):
            # 이미지의 일부분 추출
            region = image[i:i+kernel_height, j:j+kernel_width]
            # 필터와 곱한 후 합계
            feature_map[i, j] = np.sum(region * kernel)

    return feature_map


# 1. 업로드된 강아지 이미지 불러오기
print("🐕 이미지 필터 적용 데모\n")

# 업로드된 이미지 불러오기
try:
    dog_image = Image.open('/content/강아지.JPG')
    # 그레이스케일로 변환
    dog_image_gray = dog_image.convert('L')
    # numpy 배열로 변환
    test_image = np.array(dog_image_gray)
    print(f"✅ 강아지 이미지 로드 완료! 크기: {test_image.shape}")
except Exception as e:
    print(f"⚠️ 이미지 로드 실패: {e}")
    print("대체 이미지 사용")
    # 대체 이미지 생성
    test_image = np.array([
        [50, 50, 50, 50, 50, 50, 50, 50],
        [50, 100, 100, 100, 100, 100, 100, 50],
        [50, 100, 200, 200, 200, 200, 100, 50],
        [50, 100, 200, 255, 255, 200, 100, 50],
        [50, 100, 200, 255, 255, 200, 100, 50],
        [50, 100, 200, 200, 200, 200, 100, 50],
        [50, 100, 100, 100, 100, 100, 100, 50],
        [50, 50, 50, 50, 50, 50, 50, 50]
    ])

# 2. 필터 정의 (예시의 3x3 필터)
# 엣지 검출 필터 (Edge Detection Filter)
edge_filter = np.array([
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
])

print("📊 필터 (Filter/Kernel):")
print(edge_filter)
print()

# 3. 컨볼루션 적용
feature_map = apply_convolution(test_image, edge_filter)

print("✅ 컨볼루션 연산 완료!")
print(f"입력 이미지 크기: {test_image.shape}")
print(f"필터 크기: {edge_filter.shape}")
print(f"특성 맵 크기: {feature_map.shape}")
print()

# 4. 시각화
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

# 원본 이미지
axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('Input Image\n(Original)', fontsize=14, fontweight='bold')
axes[0].axis('off')
axes[0].text(0.5, -0.1, f'Size: {test_image.shape[0]}x{test_image.shape[1]}',
             ha='center', transform=axes[0].transAxes)

# 필터
axes[1].imshow(edge_filter, cmap='RdBu', vmin=-1, vmax=8)
axes[1].set_title('Filter (Kernel)', fontsize=14, fontweight='bold')
axes[1].axis('off')
# 필터 값 표시
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, str(edge_filter[i, j]),
                    ha='center', va='center', fontsize=16, fontweight='bold')
axes[1].text(0.5, -0.1, f'Size: 3x3',
             ha='center', transform=axes[1].transAxes)

# 특성 맵
im = axes[2].imshow(feature_map, cmap='gray')
axes[2].set_title('Feature Map\n(After Convolution)', fontsize=14, fontweight='bold')
axes[2].axis('off')
axes[2].text(0.5, -0.1, f'Size: {feature_map.shape[0]}x{feature_map.shape[1]}',
             ha='center', transform=axes[2].transAxes)

plt.colorbar(im, ax=axes[2], fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig('/content/convolution_demo.png', dpi=300, bbox_inches='tight')
print("✅ 이미지 저장 완료: convolution_demo.png\n")

# 5. 다양한 필터 예시
print("="*60)
print("📚 다양한 필터 예시")
print("="*60)

# 여러 종류의 필터 정의
filters = {
    'Edge Detection': np.array([[-1, -1, -1],
                                [-1,  8, -1],
                                [-1, -1, -1]]),

    'Blur (Average)': np.array([[1/9, 1/9, 1/9],
                                [1/9, 1/9, 1/9],
                                [1/9, 1/9, 1/9]]),

    'Sharpen': np.array([[ 0, -1,  0],
                        [-1,  5, -1],
                        [ 0, -1,  0]]),

    'Vertical Edge': np.array([[-1, 0, 1],
                               [-1, 0, 1],
                               [-1, 0, 1]]),

    'Horizontal Edge': np.array([[-1, -1, -1],
                                 [ 0,  0,  0],
                                 [ 1,  1,  1]])
}

# 각 필터 적용 결과 시각화
fig, axes = plt.subplots(2, 3, figsize=(10, 5))
axes = axes.flatten()

# 원본 이미지
axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0].axis('off')

# 각 필터 적용
for idx, (filter_name, filter_kernel) in enumerate(filters.items(), 1):
    result = apply_convolution(test_image, filter_kernel)
    axes[idx].imshow(result, cmap='gray')
    axes[idx].set_title(filter_name, fontsize=12, fontweight='bold')
    axes[idx].axis('off')

    print(f"\n{filter_name}:")
    print(filter_kernel)

plt.tight_layout()
plt.savefig('', dpi=300, bbox_inches='tight')
print("\n✅ 다양한 필터 결과 저장 완료: various_filters.png")

# 6. 컨볼루션 과정 상세 설명
print("\n" + "="*60)
print("🔍 컨볼루션 연산 과정 상세 설명")
print("="*60)

# 작은 예시로 단계별 설명
small_image = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
])

simple_filter = np.array([
    [1, 0],
    [0, 1]
])

print("\n입력 이미지 (4x4):")
print(small_image)
print("\n필터 (2x2):")
print(simple_filter)
print("\n컨볼루션 과정:")

output_h = small_image.shape[0] - simple_filter.shape[0] + 1
output_w = small_image.shape[1] - simple_filter.shape[1] + 1
result = np.zeros((output_h, output_w))

for i in range(output_h):
    for j in range(output_w):
        region = small_image[i:i+2, j:j+2]
        value = np.sum(region * simple_filter)
        result[i, j] = value
        print(f"\n위치 ({i},{j}):")
        print(f"이미지 영역:\n{region}")
        print(f"필터와 곱셈: {region.flatten()} * {simple_filter.flatten()}")
        print(f"합계: {value}")

print("\n최종 특성 맵 (Feature Map):")
print(result.astype(int))

print("\n" + "="*60)
print("🎉 데모 완료!")
print("="*60)

#맥스 풀링의 효과(영상의 크기를 축소해서 계산량 감소의 효과)
## 2X2  75% 감소
## 4X4  94% 감소
## 8X8  98% 감소

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# 맥스풀링 함수
def max_pooling(image, pool_size=2, stride=2):
    """
    이미지에 맥스풀링을 적용하는 함수

    Parameters:
    - image: 입력 이미지 (2D numpy array)
    - pool_size: 풀링 윈도우 크기 (기본값: 2x2)
    - stride: 이동 간격 (기본값: 2)

    Returns:
    - pooled_image: 맥스풀링 적용 후 이미지
    """
    image_height, image_width = image.shape

    # 출력 이미지 크기 계산
    output_height = (image_height - pool_size) // stride + 1
    output_width = (image_width - pool_size) // stride + 1

    # 풀링된 이미지 초기화
    pooled_image = np.zeros((output_height, output_width))

    # 맥스풀링 연산
    for i in range(output_height):
        for j in range(output_width):
            # 시작 위치 계산
            start_i = i * stride
            start_j = j * stride

            # 풀링 영역 추출
            pool_region = image[start_i:start_i+pool_size, start_j:start_j+pool_size]

            # 최대값 선택
            pooled_image[i, j] = np.max(pool_region)

    return pooled_image


# 평균풀링 함수 (비교용)
def average_pooling(image, pool_size=2, stride=2):
    """
    이미지에 평균풀링을 적용하는 함수
    """
    image_height, image_width = image.shape

    output_height = (image_height - pool_size) // stride + 1
    output_width = (image_width - pool_size) // stride + 1

    pooled_image = np.zeros((output_height, output_width))

    for i in range(output_height):
        for j in range(output_width):
            start_i = i * stride
            start_j = j * stride
            pool_region = image[start_i:start_i+pool_size, start_j:start_j+pool_size]
            pooled_image[i, j] = np.mean(pool_region)

    return pooled_image


print("🏊 맥스풀링(Max Pooling) 데모\n")

# 1. 강아지 이미지 불러오기
try:
    dog_image = Image.open('/content/강아지.JPG')
    dog_image_gray = dog_image.convert('L')
    original_image = np.array(dog_image_gray)
    print(f"✅ 강아지 이미지 로드 완료! 크기: {original_image.shape}")
except Exception as e:
    print(f"⚠️ 이미지 로드 실패: {e}")
    exit()

# 2. 다양한 풀링 크기로 실험
pool_sizes = [2, 4, 8]
results = {}

print("\n" + "="*60)
print("📊 다양한 풀링 크기 적용")
print("="*60)

for pool_size in pool_sizes:
    max_pooled = max_pooling(original_image, pool_size=pool_size, stride=pool_size)
    avg_pooled = average_pooling(original_image, pool_size=pool_size, stride=pool_size)

    results[pool_size] = {
        'max': max_pooled,
        'avg': avg_pooled
    }

    print(f"\n풀링 크기 {pool_size}x{pool_size}:")
    print(f"  원본: {original_image.shape} → 맥스풀링 후: {max_pooled.shape}")
    print(f"  크기 감소율: {(1 - max_pooled.size / original_image.size) * 100:.1f}%")

# 3. 시각화 - 맥스풀링 vs 평균풀링
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# 원본 이미지
axes[0, 0].imshow(original_image, cmap='gray')
axes[0, 0].set_title(f'Original Image\n{original_image.shape[0]}x{original_image.shape[1]}',
                     fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[1, 0].imshow(original_image, cmap='gray')
axes[1, 0].set_title(f'Original Image\n{original_image.shape[0]}x{original_image.shape[1]}',
                     fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

# 각 풀링 크기별 결과
for idx, pool_size in enumerate(pool_sizes, 1):
    # 맥스풀링
    axes[0, idx].imshow(results[pool_size]['max'], cmap='gray')
    axes[0, idx].set_title(f'Max Pooling {pool_size}x{pool_size}\n{results[pool_size]["max"].shape[0]}x{results[pool_size]["max"].shape[1]}',
                          fontsize=12, fontweight='bold', color='red')
    axes[0, idx].axis('off')

    # 평균풀링
    axes[1, idx].imshow(results[pool_size]['avg'], cmap='gray')
    axes[1, idx].set_title(f'Average Pooling {pool_size}x{pool_size}\n{results[pool_size]["avg"].shape[0]}x{results[pool_size]["avg"].shape[1]}',
                          fontsize=12, fontweight='bold', color='blue')
    axes[1, idx].axis('off')

plt.suptitle('Max Pooling vs Average Pooling Comparison', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig('/content/max_pooling_comparison.png', dpi=300, bbox_inches='tight')
print("\n✅ 비교 이미지 저장 완료: max_pooling_comparison.png")

# 4. 맥스풀링 과정 상세 시각화
print("\n" + "="*60)
print("🔍 맥스풀링 과정 상세 설명")
print("="*60)

# 작은 영역으로 상세 설명
small_region = original_image[50:58, 50:58]  # 8x8 영역 추출

print("\n원본 8x8 영역:")
print(small_region)

# 2x2 맥스풀링 적용
pooled_small = max_pooling(small_region, pool_size=2, stride=2)

print("\n2x2 맥스풀링 적용 과정:")
print("-" * 60)

for i in range(0, 8, 2):
    for j in range(0, 8, 2):
        region = small_region[i:i+2, j:j+2]
        max_val = np.max(region)
        print(f"\n위치 ({i}:{i+2}, {j}:{j+2}):")
        print(region)
        print(f"→ 최대값: {max_val}")

print("\n맥스풀링 결과 (4x4):")
print(pooled_small.astype(int))

# 5. 상세 시각화 - 맥스풀링 과정
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 원본 영역
im1 = axes[0].imshow(small_region, cmap='viridis', interpolation='nearest')
axes[0].set_title('Original 8x8 Region', fontsize=14, fontweight='bold')
axes[0].grid(True, which='both', color='white', linewidth=2)
axes[0].set_xticks(np.arange(-0.5, 8, 1))
axes[0].set_yticks(np.arange(-0.5, 8, 1))
axes[0].set_xticklabels([])
axes[0].set_yticklabels([])

# 값 표시
for i in range(8):
    for j in range(8):
        axes[0].text(j, i, f'{int(small_region[i, j])}',
                    ha='center', va='center', color='white', fontsize=8)

plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)

# 풀링 윈도우 표시
axes[1].imshow(small_region, cmap='viridis', interpolation='nearest')
axes[1].set_title('2x2 Pooling Windows', fontsize=14, fontweight='bold')

# 2x2 윈도우 구분선 그리기
for i in range(0, 8, 2):
    axes[1].axhline(y=i-0.5, color='red', linewidth=3)
    axes[1].axvline(x=i-0.5, color='red', linewidth=3)
axes[1].axhline(y=7.5, color='red', linewidth=3)
axes[1].axvline(x=7.5, color='red', linewidth=3)

axes[1].set_xticks(np.arange(-0.5, 8, 1))
axes[1].set_yticks(np.arange(-0.5, 8, 1))
axes[1].set_xticklabels([])
axes[1].set_yticklabels([])

# 맥스풀링 결과
im3 = axes[2].imshow(pooled_small, cmap='viridis', interpolation='nearest')
axes[2].set_title('Max Pooled 4x4 Result', fontsize=14, fontweight='bold')
axes[2].grid(True, which='both', color='white', linewidth=2)
axes[2].set_xticks(np.arange(-0.5, 4, 1))
axes[2].set_yticks(np.arange(-0.5, 4, 1))
axes[2].set_xticklabels([])
axes[2].set_yticklabels([])

# 값 표시
for i in range(4):
    for j in range(4):
        axes[2].text(j, i, f'{int(pooled_small[i, j])}',
                    ha='center', va='center', color='white', fontsize=10, fontweight='bold')

plt.colorbar(im3, ax=axes[2], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.savefig('/content/max_pooling_process.png', dpi=300, bbox_inches='tight')
print("✅ 과정 상세 이미지 저장 완료: max_pooling_process.png")

# 6. 전체 프로세스: 컨볼루션 + 맥스풀링
print("\n" + "="*60)
print("🔄 CNN 프로세스: 컨볼루션 → 맥스풀링")
print("="*60)

# 엣지 검출 필터
edge_filter = np.array([
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
])

# 컨볼루션 함수 (간단 버전)
def simple_convolution(image, kernel):
    image_h, image_w = image.shape
    kernel_h, kernel_w = kernel.shape

    output_h = image_h - kernel_h + 1
    output_w = image_w - kernel_w + 1

    output = np.zeros((output_h, output_w))

    for i in range(output_h):
        for j in range(output_w):
            region = image[i:i+kernel_h, j:j+kernel_w]
            output[i, j] = np.sum(region * kernel)

    return output

# 전체 프로세스 적용
print("\n단계 1: 컨볼루션 적용")
conv_result = simple_convolution(original_image, edge_filter)
print(f"  원본: {original_image.shape} → 컨볼루션 후: {conv_result.shape}")

# 음수값 처리 (ReLU)
conv_result_relu = np.maximum(conv_result, 0)
print(f"\n단계 2: ReLU 활성화 함수 (음수 제거)")

print(f"\n단계 3: 맥스풀링 적용")
final_result = max_pooling(conv_result_relu, pool_size=2, stride=2)
print(f"  컨볼루션 후: {conv_result_relu.shape} → 맥스풀링 후: {final_result.shape}")

# 전체 프로세스 시각화
fig, axes = plt.subplots(1, 4, figsize=(18, 5))

axes[0].imshow(original_image, cmap='gray')
axes[0].set_title(f'Step 1: Original\n{original_image.shape[0]}x{original_image.shape[1]}',
                 fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(conv_result_relu, cmap='gray')
axes[1].set_title(f'Step 2: Convolution + ReLU\n{conv_result_relu.shape[0]}x{conv_result_relu.shape[1]}',
                 fontsize=12, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(final_result, cmap='gray')
axes[2].set_title(f'Step 3: Max Pooling (2x2)\n{final_result.shape[0]}x{final_result.shape[1]}',
                 fontsize=12, fontweight='bold')
axes[2].axis('off')

axes[3].imshow(max_pooling(final_result, pool_size=2, stride=2), cmap='gray')
axes[3].set_title(f'Step 4: Max Pooling Again\n{max_pooling(final_result, pool_size=2, stride=2).shape[0]}x{max_pooling(final_result, pool_size=2, stride=2).shape[1]}',
                 fontsize=12, fontweight='bold')
axes[3].axis('off')

plt.suptitle('Complete CNN Process: Convolution → ReLU → Max Pooling',
             fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig('/content/cnn_complete_process.png', dpi=300, bbox_inches='tight')
print("\n✅ 전체 프로세스 이미지 저장 완료: cnn_complete_process.png")

# 7. 맥스풀링의 효과 요약
print("\n" + "="*60)
print("📈 맥스풀링의 효과")
print("="*60)
print("\n1. 차원 축소 (Dimensionality Reduction):")
print(f"   - 원본: {original_image.shape} = {original_image.size:,} 픽셀")
print(f"   - 2x2 풀링: {results[2]['max'].shape} = {results[2]['max'].size:,} 픽셀 (75% 감소)")
print(f"   - 4x4 풀링: {results[4]['max'].shape} = {results[4]['max'].size:,} 픽셀 (93.75% 감소)")
print(f"   - 8x8 풀링: {results[8]['max'].shape} = {results[8]['max'].size:,} 픽셀 (98.44% 감소)")

print("\n2. 계산량 감소:")
print("   - 파라미터 수가 줄어들어 학습 속도 향상")
print("   - 과적합(Overfitting) 방지")

print("\n3. 위치 불변성 (Translation Invariance):")
print("   - 특징의 정확한 위치가 아닌 존재 여부에 집중")
print("   - 이미지가 약간 이동해도 같은 특징 인식")

print("\n4. 중요한 특징 강조:")
print("   - 맥스풀링: 가장 강한 특징(최대값) 선택")
print("   - 평균풀링: 전체적인 정보 유지")

print("\n" + "="*60)
print("🎉 맥스풀링 데모 완료!")
print("="*60)

## 다항식 미분을 활용한 인공지능 활용
##1. 1차 미분만을 사용해 함수를 최정화시키자. 경사하강법

In [ ]:
## 파이썬으로 미분할 함수와 미분할 변수를 이용하여 미분계산하기
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
##
x = sym.Symbol('x')
y = 2*x**3 + 3*x**2 + x + 1
first_diff_y = sym.diff(y, x )
second_diff_y = sym.diff(y, x,2)

print(f"일차미분함수 : {first_diff_y}, 이차미분함수: {second_diff_y}")

In [ ]:
# 임계점 찾기 (1차 도함수 = 0인 점)
critical_points = sym.solve(first_diff_y, x)
print(f"임계점 : {critical_points}")


In [ ]:
##특성화맵

